In [7]:
import numpy as np
from datetime import timedelta
from scipy import stats

def absolute_difference(data, simulation):
    return np.abs(data - simulation)


class ABCSMC:

    def __init__(self, 
                 model, 
                 priors, 
                 parameters, 
                 observed_data, 
                 perturbation_kernel, 
                 distance_function = absolute_difference, 
                 p_discrete_transition : float = 0.3,
                 num_particles : int = 1000, 
                 max_generations : int = 10, 
                 minimum_epsilon : float = 0.15, 
                 max_walltime : timedelta = None,
                 include_quantiles : bool = True, 
                 dates_column="simulation_dates"):
        self.model = model
        self.priors = priors
        self.parameters = parameters
        self.observed_data = observed_data
        self.perturbation_kernel = perturbation_kernel
        self.distance_function = distance_function
        self.p_discrete_transition = p_discrete_transition
        self.num_particles = num_particles
        self.max_generations = max_generations
        self.minimum_epsilon = minimum_epsilon
        self.max_walltime = max_walltime
        self.include_quantiles = include_quantiles
        self.dates_column = dates_column
        self.particles = None
        self.weights = None
        self.tolerance = None

    def initialize_particles(self):
        # Initialize particles from the prior
        self.particles = [{p: priors[p].rvs() for p in priors} for _ in range(self.num_particles)]
        self.weights = np.ones(self.num_particles) / self.num_particles
        self.tolerance = np.inf

    def run(self):

        # get continuous and discrete parameters
        continuous_params, discrete_params = [], []
        for param in self.priors:
            if isinstance(self.priors[param].dist, stats.rv_continuous):
                continuous_params.append(param)
            elif isinstance(self.priors[param].dist, stats.rv_discrete):
                discrete_params.append(param)

        self.initialize_particles()

        for generation in range(self.max_generations):
            # compute covariance matrix
            cov_matrix = compute_covariance_matrix(self.particles, continuous_params)
            
            new_particles = []
            new_weights = []
            distances = []

            total_accepted = 0
            #for i in range(self.num_particles):
            while total_accepted < self.num_particles:
                # Perturb particle
                i = np.random.choice(range(self.num_particles), p=self.weights)
                #particle = self.perturbation_kernel(self.particles[i])
                particle = perturbation_kernel(self.particles[i], continuous_params, discrete_params, cov_matrix, priors, self.p_discrete_transition)

                # Simulate data from perturbed particle
                full_params = {}
                full_params.update(particle)
                full_params.update(self.parameters)
                simulated_data = self.model(full_params)

                # Calculate distance
                distance = self.distance_function(self.observed_data, simulated_data)
                if distance < self.tolerance:
                    new_particles.append(particle)
                    distances.append(distance)
                    #new_weights.append(self.weights[i])  # Assuming uniform weights for simplicity
                    total_accepted += 1

            # Normalize weights
            #new_weights = np.array(new_weights)
            #new_weights /= np.sum(new_weights)

            # Update particles and weights
            self.weights = compute_weights(new_particles, self.particles, self.weights, priors, cov_matrix, continuous_params, discrete_params, self.p_discrete_transition)
            self.particles = new_particles

            # Update tolerance for the next generation
            self.tolerance = np.percentile(distances, 75)  # Median distance

            print(f"Generation {generation + 1}: Tolerance = {self.tolerance}, Accepted Particles = {len(new_particles)}")

            if len(new_particles) == 0:
                print("No particles accepted in this generation. Stopping.")
                break

        return self.particles, self.weights

# Example usage:

# Define a simple model, prior_sampler, perturbation_kernel, and distance_function

def model(parameters):
    # Simple model example, e.g., particle directly represents simulated data
    return parameters["beta"] + parameters["mu"]


def perturbation_kernel(particle, continuous_params, discrete_params, cov_matrix, priors, p_discrete_transition):
    """
    Perturbs a particle's parameters.

    Parameters:
    - particle: dict, keys are parameter names, values are current values of the parameters
    - continuous_params: list of parameter names that are continuous
    - discrete_params: list of parameter names that are discrete
    - cov_matrix: dict, covariance matrices for the continuous parameters
    - priors: dict, prior distributions for the parameters
    - discrete_jump_probs: dict, probability distributions for the discrete jumps

    Returns:
    - perturbed_particle: dict, the perturbed particle
    """
    perturbed_particle = particle.copy()

    # Perturb continuous parameters
    if continuous_params:
        # Extract current values for continuous parameters
        current_values = np.array([particle[param] for param in continuous_params])

        # Apply multivariate normal transition
        perturbed_values = np.random.multivariate_normal(
            mean=np.array(current_values),
            cov=cov_matrix
        )

        # Update the perturbed particle with new continuous values
        for i, param in enumerate(continuous_params):
            perturbed_particle[param] = perturbed_values[i]

    # Perturb discrete parameters
    for param in discrete_params:
    
        current_value = particle[param]
        # The probability distribution must exclude the current value if we are jumping to a different state
        if np.random.rand() < p_discrete_transition:
            # Get the probability distribution for the current parameter
            perturbed_particle[param] = np.random.choice(range(priors[param].support()[0], priors[param].support()[1]))

    return perturbed_particle


def compute_covariance_matrix(particles, continuous_params):
    """
    Computes the covariance matrix for the continuous parameters in a list of particles.

    Parameters:
    - particles: list of dicts, each dict contains a set of parameters
    - continuous_params: list of str, the names of the continuous parameters

    Returns:
    - covariance_matrix: numpy array, the covariance matrix of the continuous parameters
    """
    # Extract the values of the continuous parameters from each particle
    data = np.array([[particle[param] for param in continuous_params] for particle in particles])

    # Compute the covariance matrix
    if len(continuous_params) == 1:
        covariance_matrix = np.array([[np.var(data)]])
    else:
        covariance_matrix = np.cov(data, rowvar=False)

    return covariance_matrix


import numpy as np
from scipy.stats import multivariate_normal

def compute_weights(new_particles, old_particles, old_weights, priors, cov_matrix, continuous_params, discrete_params, discrete_transition_prob):
    """
    Computes the weights for new particles in generation t.
    
    Parameters:
    - new_particles: list of dicts, the particles in the current generation t
    - old_particles: list of dicts, the particles from the previous generation t-1
    - old_weights: list of floats, the weights of particles from generation t-1
    - priors: dict, keys are parameter names and values are scipy.stats objects representing the priors
    - cov_matrix: covariance matrix used in the perturbation kernel for continuous parameters
    - continuous_params: list of parameter names that are continuous
    - discrete_params: list of parameter names that are discrete
    - discrete_transition_prob: float, probability of any transition for discrete parameters
    
    Returns:
    - new_weights: list of floats, the computed weights for the new particles
    """
    # Precompute inverse and determinant of covariance matrix for the multivariate normal
    #inv_cov_matrix = np.linalg.inv(cov_matrix)
    #log_det_cov_matrix = np.log(np.linalg.det(cov_matrix))

    new_weights = np.zeros(len(new_particles))
    
    for i, new_particle in enumerate(new_particles):
        # Compute the prior probability of the new particle
        prior_prob = np.prod([priors[param].pdf(new_particle[param]) for param in continuous_params])
        prior_prob *= np.prod([priors[param].pmf(new_particle[param]) for param in discrete_params])

        # Compute the denominator of the weight expression
        diff_vectors = np.array([[new_particle[param] - old_particle[param] for param in continuous_params] for old_particle in old_particles])
        continuous_kernel_probs = multivariate_normal.logpdf(diff_vectors, mean=np.zeros(len(continuous_params)), cov=cov_matrix, allow_singular=True)

        discrete_kernel_probs = np.ones(len(old_particles))
        for param in discrete_params:
            transitions = np.array([new_particle[param] != old_particle[param] for old_particle in old_particles])
            discrete_kernel_probs *= np.where(transitions, discrete_transition_prob, 1 - discrete_transition_prob)

        kernel_probs = np.exp(continuous_kernel_probs) * discrete_kernel_probs
        denominator = np.sum(old_weights * kernel_probs)
        
        # Calculate the weight for the new particle
        new_weights[i] = prior_prob / denominator

    # Normalize the weights so they sum to 1
    new_weights /= np.sum(new_weights)
    
    return new_weights


priors = {"beta": stats.uniform(0.05, 0.3)}
parameters = {"mu": 0.2}
observed_data = 0.3

abc_smc = ABCSMC(model, 
                 priors, 
                 parameters, 
                 observed_data, 
                 perturbation_kernel, 
                 distance_function = absolute_difference)


final_particles, final_weights = abc_smc.run()

Generation 1: Tolerance = 0.18814751212193076, Accepted Particles = 1000
Generation 2: Tolerance = 0.1309726971134102, Accepted Particles = 1000
Generation 3: Tolerance = 0.09167672716788353, Accepted Particles = 1000
Generation 4: Tolerance = 0.06330579682749779, Accepted Particles = 1000
Generation 5: Tolerance = 0.04412964593606952, Accepted Particles = 1000
Generation 6: Tolerance = 0.031611003598288986, Accepted Particles = 1000
Generation 7: Tolerance = 0.023142908731751816, Accepted Particles = 1000
Generation 8: Tolerance = 0.016502317483896287, Accepted Particles = 1000
Generation 9: Tolerance = 0.011838817000579718, Accepted Particles = 1000
Generation 10: Tolerance = 0.008318342551507138, Accepted Particles = 1000


In [13]:
import pandas as pd

posterior_dict = {}
for param in priors.keys(): 
    posterior_dict[param] = [sample[param] for sample in final_particles]


posterior_df = pd.DataFrame(posterior_dict) 
posterior_df

,beta,weights
0,0.111334,0.001224
1,0.095103,0.000951
2,0.094481,0.000961
3,0.098274,0.000919
4,0.111254,0.001218
...,...,...
995,0.099950,0.000917
996,0.097460,0.000923
997,0.090750,0.001070
998,0.090916,0.001064


In [14]:
np.percentile(np.arange(0, 100), 75) 

74.25

In [15]:
np.quantile(np.arange(0, 100), 0.75) 

74.25

In [ ]:
from scipy.stats import multivariate_normal
import scipy



# Example usage:

# Assuming priors, old_particles, old_weights, cov_matrix, continuous_params, and discrete_params are defined

priors = {
    'param1': scipy.stats.uniform(0, 1),  # Continuous: Uniform prior between 0 and 1
    'param2': scipy.stats.norm(0.5, 0.1),  # Continuous: Normal prior with mean 0.5 and std 0.1
    'param3': scipy.stats.randint(1, 4),  # Discrete: Discrete uniform prior over [1, 2, 3]
    'param4': scipy.stats.bernoulli(0.6)  # Discrete: Bernoulli prior with p=0.6
}

old_particles = [
    {'param1': 0.5, 'param2': 0.3, 'param3': 2, 'param4': 1},
    {'param1': 0.6, 'param2': 0.4, 'param3': 3, 'param4': 0},
    {'param1': 0.4, 'param2': 0.2, 'param3': 1, 'param4': 1}
]

old_weights = [0.3, 0.4, 0.3]

new_particles = [
    {'param1': 0.55, 'param2': 0.35, 'param3': 2, 'param4': 1},
    {'param1': 0.45, 'param2': 0.25, 'param3': 1, 'param4': 0}
]

continuous_params = ['param1', 'param2']
discrete_params = ['param3', 'param4']

cov_matrix = np.array([
    [0.01, 0],   # Covariance matrix for continuous parameters param1 and param2
    [0, 0.01]
])

discrete_transition_prob = 0.1  # Example probability of transitioning between any two discrete states

new_weights = compute_weights(new_particles, old_particles, old_weights, priors, cov_matrix, continuous_params, discrete_params, discrete_transition_prob)
print("New Weights:", new_weights)


In [5]:
import scipy 

priors = {
    'param1': scipy.stats.uniform(0, 1),  # Continuous: Uniform prior between 0 and 1
    'param2': scipy.stats.norm(0.5, 0.1),  # Continuous: Normal prior with mean 0.5 and std 0.1
    'param3': scipy.stats.randint(1, 4),  # Discrete: Discrete uniform prior over [1, 2, 3]
    'param4': scipy.stats.bernoulli(0.6)  # Discrete: Bernoulli prior with p=0.6
}

old_particles = [
    {'param1': 0.5, 'param2': 0.3, 'param3': 2, 'param4': 1},
    {'param1': 0.6, 'param2': 0.4, 'param3': 3, 'param4': 0},
    {'param1': 0.4, 'param2': 0.2, 'param3': 1, 'param4': 1}
]

old_weights = [0.3, 0.4, 0.3]

new_particles = [
    {'param1': 0.55, 'param2': 0.35, 'param3': 2, 'param4': 1},
    {'param1': 0.45, 'param2': 0.25, 'param3': 1, 'param4': 0}
]

continuous_params = ['param1', 'param2']
discrete_params = ['param3', 'param4']

cov_matrix = np.array([
    [0.01, 0],   # Covariance matrix for continuous parameters param1 and param2
    [0, 0.01]
])

discrete_transition_prob = 0.1  # Example probability of transitioning between any two discrete states

new_weights = compute_weights(new_particles, old_particles, old_weights, priors, cov_matrix, continuous_params, discrete_params, discrete_transition_prob)
print("New Weights:", new_weights)

New Weights: [0.60660834 0.39339166]
